# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [86]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import prettytable

#### Creating list of filepaths to process original event csv data files

In [87]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/minhquang/Coding/DE/nanodegree/data-modeling-with-apache-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [88]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [89]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [90]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [91]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_modeling 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [92]:
try:
    session.set_keyspace('data_modeling')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4
####  In order to execute this query, we need to have information about the sessions and the songs listened to, the table will be composed of the following columns: sessionId, itemInSession, artist, song and length. The PRIMARY KEY is composed by sessionId as partion key and itemInSession as the clustering column. This allows to have an unique key that responds to the query requirements

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
#### In order to execute this query, we need to have information about the sessions, the user id and the songs listened to including the artist information, the table will be composed of the following columns: userId, sessionId, itemInSession, artist and song. The PRIMARY KEY is composed by userId and sessionId which is indeed a composite partition key, and itemInSession as the clustering column. This allows to have an unique key and a clustering key responding to the requirements
 

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#### In order to execute this query, we need to have information about user's information and the songs listened to, the table will be composed of the following columns: song, firstName, lastName and userId. The PRIMARY KEY is composed by song, and userId as the clustering column. This allows to have an unique key responding to the requirements



In [93]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_session_and_item"
query = query + "(session_id int, item_in_session int, artist text, length decimal, song text, PRIMARY KEY(session_id, item_in_session))"

try:
    session.execute(query)

except Exception as e:
    print(e)

In [94]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session_and_item (session_id, item_in_session, artist, first_name, last_name, length, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[4], float(line[5]), line[9]))

In [95]:
from prettytable import PrettyTable

query = "SELECT artist, song, length FROM song_info_by_session_and_item WHERE session_id = 338 and item_in_session = 4"
table = PrettyTable()
table.field_names = ["artist", "song", "length"]

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    table.add_row([row.artist, row.song, row.length])

print(table)

+-----------+---------------------------------+----------+
|   artist  |               song              |  length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


#### At sessionId=338 and itemInsession=4, the artist was 'Faithless', and the song was 'Music Matters (Mark Knight Dub)' with a length of 495.307312 seconds.

In [96]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_id_and_sessionId"
query = query + " (user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"

try:
    session.execute(query)

except Exception as e:
    print(e)

In [97]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_id_and_sessionId (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [98]:
query = "CREATE TABLE IF NOT EXISTS user_info_by_song"
query = query + " (song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"

try:
    session.execute(query)

except Exception as e:
    print(e)

In [99]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO user_info_by_song (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [104]:
from prettytable import PrettyTable

query = "SELECT item_in_session, artist, song, first_name, last_name FROM song_info_by_id_and_sessionId WHERE user_id=10 AND session_id=182"
table = PrettyTable()
table.field_names = ["item_in_session", "artist", "song", "first_name", "last_name"]

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    table.add_row([row.item_in_session, row.artist, row.song, row.first_name, row.last_name])         

print(table)    

+-----------------+-------------------+------------------------------------------------------+------------+-----------+
| item_in_session |       artist      |                         song                         | first_name | last_name |
+-----------------+-------------------+------------------------------------------------------+------------+-----------+
|        0        |  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|        1        |    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
|        2        | Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|        3        |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-----------------+-------------------+------------------------------------------------------+------------+-----------+


#### In sessionId=182, userId=10, Sylvie Cruz listened to 4 songs: 'Keep on Keepin' On' by 'Down To The Bone', 'Greece 2000' by 'Three Drives', 'Kilometer' by 'Sebastien Tellier', and 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)' by 'Lonnie Gordon'.

In [101]:
query = "CREATE TABLE IF NOT EXISTS user_info_by_song"
query = query + " (song text, first_name text, last_name text, user_id int, PRIMARY KEY (song, user_id))"

try:
    session.execute(query)

except Exception as e:
    print(e)
                    

In [102]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO user_info_by_song (song, first_name, last_name, user_id)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [103]:
from prettytable import PrettyTable

query = "SELECT first_name, last_name FROM user_info_by_song WHERE song='All Hands Against His Own'"
table = PrettyTable()
table.field_names = ["first_name", "last_name"]

try:
    rows = session.execute(query)

except Exception as e:
    print(e)

for row in rows:
    table.add_row([row.first_name, row.last_name])
print(table)    

+------------+-----------+
| first_name | last_name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


#### Three users listened to the song 'All Hands Against His Own': Jacqueline Lynch, Sara Johnson, and Tegan Levine.

### Drop the tables before closing out the sessions

In [105]:
query = "drop table song_info_by_session_and_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_info_by_id_and_sessionId"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_info_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [106]:
session.shutdown()
cluster.shutdown()